<a href="https://colab.research.google.com/github/Foco22/AFPs/blob/master/AnalisisAFPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bs4

In [2]:
!pip install requests

In [43]:
!pip install beautifulsoup4

In [7]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
G

In [59]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# **Rentabilidad Mensuales de las AFPs**

In [61]:
!pip install selenium-wire

     |████████████████████████████████| 839kB 3.4MB/s 


In [73]:
! pip3 install  pyvirtualdisplay selenium webdriver_manager  > /dev/null
! apt install firefox  xvfb > /dev/null

In [99]:
import bs4 as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)


driver.get('https://www.spensiones.cl/apps/rentabilidad/getRentabilidad.php?tiprent=FP&template=0')

wait = WebDriverWait(driver, 10)

html = driver.execute_script('return document.documentElement.outerHTML')
soup = bs.BeautifulSoup(html, 'html.parser')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [100]:
def cambio_url(ano,mes):
    
    s2 = Select(driver.find_element_by_name('aaaa'))
    s2.select_by_value(ano)
    s2 = Select(driver.find_element_by_name('mm'))
    s2.select_by_value(mes)
    s3 = driver.find_element_by_name('btn')
    s3.click()

    #soup = BeautifulSoup(driver.page_source)
    #soup = bs4.BeautifulSoup(web_page_text, "lxml")
    
    html = driver.execute_script('return document.documentElement.outerHTML')
    soup = bs.BeautifulSoup(html, 'html.parser')
    #soup = bs(driver.content)

    tabla = soup.find_all('td',{'align':'right'})
    
    return tabla

In [104]:
df = pd.DataFrame(columns =['Año','Periodo','AFP','Fondo','Rentabilidad Mensual'])
FONDOS = ['FONDO A', 'FONDO B', 'FONDO C', 'FONDO D', 'FONDO E']
AFPS = ['CAPITAL', 'CUPRUM', 'HABITAT', 'MODELO', 'PLANVITAL', 'PROVIDA','UNO','SISTEMA']

def datos_2019_2020(df):

    
    años_lista = ['2019','2019','2019','2020','2020','2020','2020','2020','2020','2020','2020']
    mes_lista =  ['10','11','12','01','02','03','04','05','06','07','08']
    
    for a, b in zip(años_lista,mes_lista):
        
        gdp_table = cambio_url(str(a),str(b))
        valores = []
        for idx, val in enumerate(gdp_table):
                val = str(val)
                start = '"right">'
                end = '</td>'
                val = val[val.find(start)+len(start):val.rfind(end)]
                numero = ''
                for i in list(val):
                    if i not in  (',','%','-'):
                        numero = numero + i
                if numero != '\xa0':
                    if val[0] != '-':
                        numero = int(numero)/100
                    else:
                        numero = int(numero)/100*-1
                else:
                        numero = 0 
     
                fondo_c = range(64,105)
                fondo_e = range(136,176)
                #print(idx,val)
                if idx in fondo_c:
                    if idx in (64, 69, 74, 79,84, 89, 94, 99,104):
                        valores.append(numero)
                elif idx in fondo_e:
                    if idx in (136,141, 146, 151, 156, 161, 166, 171):
                        valores.append(numero)
                elif idx % 4 == 0:
                        valores.append(numero)
        contador = 0
        for i in FONDOS:
            for j in AFPS:
                new_row = {'Año': a, 'Periodo':b, 'AFP':j, 'Fondo':i, 'Rentabilidad Mensual':valores[contador]}
                df = df.append(new_row, ignore_index=True)
                contador = 1 + contador
    return df
datos_2019_2020 = datos_2019_2020(df)

In [106]:
datos_2019_2020.head()

,Año,Periodo,AFP,Fondo,Rentabilidad Mensual
0,2019,10,CAPITAL,FONDO A,1.30
1,2019,10,CUPRUM,FONDO A,1.32
2,2019,10,HABITAT,FONDO A,1.51
3,2019,10,MODELO,FONDO A,1.44
4,2019,10,PLANVITAL,FONDO A,1.34
